# Import dependencies

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, SpatialDropout2D, UpSampling2D
from keras.optimizers import RMSprop

from keras.preprocessing import image 
from keras import backend as K
import os
import numpy as np
from keras.applications.resnet50 import preprocess_input
import PIL.ImageOps    
from keras.utils import np_utils
np.random.seed(23)

# Load the Hanacaraka Dataset

In [ ]:
folder = 'hanacaraka/'

img_rows, img_cols, img_chns = 60, 80, 1         # input image dimensions
nb_classes = 20
number_of_data = 1000
corruption_level = 0.3

X_train = []
Y_train = []

for cl, fld in enumerate(os.listdir(folder)):
        for f in os.listdir(folder + fld):
                img = image.load_img(folder + fld + '/' + f, 
                                     grayscale=True, target_size=(img_rows, img_cols))     # read image
                img = PIL.ImageOps.invert(img)
                img = image.img_to_array(img)
                X_train.append(img.reshape((img_rows, img_cols, 1)))
                Y_train.append(cl)


X_train = np.array(X_train)
Y_train = np.array(Y_train)

shuffled_indices = np.random.permutation(X_train.shape[0])
X_train = X_train[shuffled_indices] 
Y_train = Y_train[shuffled_indices] 
X_test = X_train[X_train.shape[0]//2:]
X_train = X_train[:X_train.shape[0]//2]

Y_test = Y_train[Y_train.shape[0]//2:]
Y_train = Y_train[:Y_train.shape[0]//2]

print('X_train original shape:', X_train.shape)

if K.image_dim_ordering() == 'th':
    # For Theano backend
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # For TensorFlow backend
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# Split train test
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

X_train = X_train[:number_of_data]
Y_train = Y_train[:number_of_data]
X_test = X_test[:number_of_data]
Y_test = Y_test[:number_of_data]

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

## Apply noise to the image
X_train_noisy = X_train + corruption_level * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noisy = X_test + corruption_level * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)
print(X_train_noisy.shape)
print(X_test_noisy.shape)

# Visualize MNIST dataset
import matplotlib.pyplot as plt
%matplotlib inline

n = 10 
plt.figure(figsize=(10, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_train[i].reshape(img_rows, img_cols))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

import matplotlib.pyplot as plt
%matplotlib inline

n = 10  
plt.figure(figsize=(10, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_train_noisy[i].reshape(img_rows, img_cols))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Convolutional Autoencoder

## Build

In [ ]:
# Settings
pool_size = (2, 2)                  # size of pooling area for max pooling
input_shape = (img_rows, img_cols, img_chns)

model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))

model.add(Conv2D(16, (3, 3), padding='same', activation='relu'))
model.add(UpSampling2D(pool_size))
model.add(Conv2D(1, (3, 3), padding='same', activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

## Train

In [ ]:
# Hyper parameters
batch_size = 128
nb_epoch = 10

model.fit(X_train_noisy, X_train,
                epochs=nb_epoch, batch_size=batch_size, shuffle=True, verbose=1,
                validation_data=(X_test_noisy, X_test))

## Evaluate

In [ ]:
evaluation = model.evaluate(X_test_noisy, X_test, batch_size=256, verbose=1)
print('Summary: Loss over the test dataset: %.2f' % (evaluation))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Decode test images 
decoded_imgs = model.predict(X_test_noisy)

n = 10  # how many digits we will display
plt.figure(figsize=(10, 2))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test_noisy[i].reshape(img_rows, img_cols))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(img_rows, img_cols))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()